<a href="https://colab.research.google.com/github/hollyemblem/raschka-llm-from-scratch/blob/chapter-2-sliding/chapter2_data_sampling_with_sliding_window.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chapter 2

#### Data sampling with a sliding window
Implementing a data loader that fetches the input–target pairs in figure 2.12 from the training dataset using a sliding window approach.



In [1]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x7c73f1868320>)

In [2]:
from importlib.metadata import version
import tiktoken

In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [5]:
## Removing first 50 for demo purposes
enc_sample = enc_text[50:]

In [6]:
## Creating x and y variables where x is input and y is target
context_size = 4         #1
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [7]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [8]:
## Converting tokens into text
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [9]:
#implementing a data loader to iterate over input dataset and return inputs and targets as pytorch tensors.
# input tensor -> text the llm sees
# target sensor -> targets for llm to predict
# code will use the token IDS directly, as BPE tokenizer performs tokenisation & conversion to token IDs in one step

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset):
    '''The GPTDatasetV1 class is based on the PyTorch Dataset class and defines how individual rows are fetched from the dataset, '''
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)    #1 Tokenizes the entire text

        for i in range(0, len(token_ids) - max_length, stride):     #2 Uses a sliding window to chunk the book into overlapping sequences of max_length
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):    #3 Returns the total number of rows in the dataset
        return len(self.input_ids)

    def __getitem__(self, idx):         #4 Returns a single row from the dataset
        return self.input_ids[idx], self.target_ids[idx]

In [13]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")                         #1 Initializes the tokenizer
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)   #2 Creates dataset
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,     #3 drop_last=True drops the last batch if it is shorter than the specified batch_size to prevent loss spikes during training.
        num_workers=num_workers     #4 The number of CPU processes to use for preprocessing
    )

    return dataloader

In [29]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=8, stride=1, shuffle=False)
data_iter = iter(dataloader)      #1 Converts dataloader into a Python iterator to fetch the next entry via Python’s built-in next() function
first_batch = next(data_iter)
print(first_batch)

##First tensor is input token IDs, second tensor is target tokenIDs

[tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])]


In [30]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]]), tensor([[ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]])]


### Understanding Stride
The stride setting dictates the number of positions the inputs shift across batches, emulating a sliding window approach. In this example above, we move the window by one position. Let's try four.

We can see examining the second batch we've taken a stride four spots, so now the input for second batch[0] is 1807.

In [31]:
dataloader2 = create_dataloader_v1(raw_text, batch_size=1, max_length=8, stride=4, shuffle=False)
data_iter2 = iter(dataloader2)      #1 Converts dataloader into a Python iterator to fetch the next entry via Python’s built-in next() function
first_batch2 = next(data_iter2)
print(first_batch2)

[tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])]


In [32]:
second_batch2 = next(data_iter2)
print(second_batch2)

[tensor([[ 1807,  3619,   402,   271, 10899,  2138,   257,  7026]]), tensor([[ 3619,   402,   271, 10899,  2138,   257,  7026, 15632]])]


Some thoughts around using batch size with strides and max length. In the book they give an example of:



```
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)
```


And note that "Note that we increase the stride to 4 to utilize the data set fully (we don’t skip a single word). This avoids any overlap between the batches since more overlap could lead to increased overfitting.". Let's print these out:


In [39]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [40]:
##Setting stride to 1 increases overlap:

In [41]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=1,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [  367,  2885,  1464,  1807],
        [ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 2885,  1464,  1807,  3619],
        [ 1464,  1807,  3619,   402],
        [ 1807,  3619,   402,   271],
        [ 3619,   402,   271, 10899],
        [  402,   271, 10899,  2138],
        [  271, 10899,  2138,   257],
        [10899,  2138,   257,  7026]])


In the above example, look how many times we see the same tokens in input:

367 is in there twice (where stride is 1 and max length is 4). 75% of the tokens seen in row 2 are from row 1. For top example, this isn't the case.

With top example we see it only once (where max length is 4 and stride is 4).